In [2]:
import proj1_helpers
import implementations
import numpy as np

In [3]:
print("Extracting dataset")
y_train_ref_source, X_train_ref_source, id1 = proj1_helpers.load_csv_data("../train.csv")
#y_test_ref, X_test_ref, id2 = proj1_helpers.load_csv_data("test.csv")


Extracting dataset


In [4]:
i = 0
X_in = []
X_out = []
y_in = []
y_out=[]
for i in range(X_train_ref_source.shape[0]):
    x = X_train_ref_source[i]
    y = y_train_ref_source[i]
    if -999 in x:
        X_in.append(x)
        y_in.append(y)
    else:
        X_out.append(x)
        y_out.append(y)
        
X_in = np.array(X_in)
X_out = np.array(X_out)
y_in = np.array(y_in)
y_out = np.array(y_out)

print(X_in.shape,X_out.shape)

X_in_train_ref, X_in_test_ref, y_in_train_ref, y_in_test_ref = implementations.split_data(X_in, y_in, 0.9,23)
X_out_train_ref, X_out_test_ref, y_out_train_ref, y_out_test_ref = implementations.split_data(X_out, y_out, 0.9,23)

(181886, 30) (68114, 30)


In [5]:
def batch_ridge_in(lambda_, degree, batch_size):
    x = []
    xt = []
    for i in range(1,degree+1):
        x.append(np.power(X_in_train_ref,i))
        xt.append(np.power(X_in_test_ref,i))
        
    X_train = np.concatenate(x,axis = 1)
    y_train = y_in_train_ref
    X_test = np.concatenate(xt,axis = 1)
    y_test = y_in_test_ref
    
    #print("X_train shape : ", X_train.shape,"    X_test.shape : ", X_test.shape)
    #print("y_train shape : ", y_train.shape,"    y_test.shape : ", y_test.shape)
    
    num_features = X_train[0].shape[0]
    #print("number of features : ", num_features)

    #print("Splitting dataset into batch")
    X_batch = np.array_split(X_train, 1)
    y_batch = np.array_split(y_train, 1)

    #print(X_batch[0].shape)
    
    w = []
    i=0
    for y,batch in zip(y_batch,X_batch):
        w.append(implementations.ridge_regression(y, batch,lambda_))
    
    
    wp =  np.sum(np.array(w),axis=0)/(len(X_batch))
    
    
    

    y_pred_ridge_train = proj1_helpers.predict_labels(wp,X_train)
    y_pred_ridge_test = proj1_helpers.predict_labels(wp,X_test)
    s_tr = 0
    s_te = 0
    tot_tr = 0
    for i,y in enumerate(y_train):
        if y == y_pred_ridge_train[i]:
            s_tr += 1
        tot_tr += 1
    tot_te = 0
    for i,y in enumerate(y_test):
        if y == y_pred_ridge_test[i]:
            s_te += 1
        tot_te += 1

    print("Ridge precision on train : ",s_tr/tot_tr,"     Ridge precision on test : ",s_te/tot_te)
    
    return wp

In [22]:
batch_ridge_in(1000,6,32)
batch_ridge_in(2000,6,32)
batch_ridge_in(3000,6,32)
batch_ridge_in(5000,6,32)
batch_ridge_in(9000,6,32)
batch_ridge_in(10000,6,32)
batch_ridge_in(20000,6,32)
batch_ridge_in(50000,6,32)
batch_ridge_in(400,6,32)

Ridge precision on train :  0.8057814132207677      Ridge precision on test :  0.809720160536588
Ridge precision on train :  0.8055676035602363      Ridge precision on test :  0.810050030238056
Ridge precision on train :  0.8055370593230176      Ridge precision on test :  0.809885095387322
Ridge precision on train :  0.8054148823741425      Ridge precision on test :  0.809720160536588
Ridge precision on train :  0.8051888550187236      Ridge precision on test :  0.8091153994172302
Ridge precision on train :  0.8052743788829362      Ridge precision on test :  0.8085106382978723
Ridge precision on train :  0.8003934097753777      Ridge precision on test :  0.8038924624773215
Ridge precision on train :  0.8030507584134101      Ridge precision on test :  0.8076309857606245
Ridge precision on train :  0.8057264335937738      Ridge precision on test :  0.8095552256858541


array([  2.29529573e-02,   1.48397478e-03,   3.82839892e-03,
        -1.05998534e-02,  -2.99965553e-02,  -3.77663147e-03,
        -6.96212299e-02,  -3.95770884e-02,   6.25119880e-03,
         9.28964260e-02,  -2.14640853e-02,  -8.47851027e-02,
         1.41653814e-02,  -3.30772896e-02,  -4.31670262e-03,
        -3.18328943e-03,  -4.77085497e-02,   2.47932915e-03,
         2.35966058e-03,  -1.44621816e-02,  -1.69092613e-03,
         1.72266563e-03,  -2.13999086e-02,   2.04642989e-02,
        -1.26892392e-03,  -2.89820644e-03,  -6.82626334e-02,
        -2.11285634e-03,   8.44783058e-03,  -9.19457447e-02,
        -1.12603631e-04,  -3.05757325e-04,  -1.25115431e-04,
         2.33962842e-04,  -2.35335866e-02,   1.59828281e-05,
         1.13750666e-03,   7.18722269e-02,  -2.44504019e-04,
         9.83039549e-05,   7.96140742e-02,   4.57773516e-02,
         8.75937653e-03,  -8.41161765e-04,   1.14072789e-02,
        -6.08292865e-03,  -1.23739543e-03,  -3.12348116e-02,
         6.88829103e-04,

In [6]:
def batch_ridge_out(lambda_, degree, batch_size):
    x = []
    xt = []
    for i in range(1,degree+1):
        x.append(np.power(X_out_train_ref,i))
        xt.append(np.power(X_out_test_ref,i))
        
    X_train = np.concatenate(x,axis = 1)
    y_train = y_out_train_ref
    X_test = np.concatenate(xt,axis = 1)
    y_test = y_out_test_ref
    
    #print("X_train shape : ", X_train.shape,"    X_test.shape : ", X_test.shape)
    #print("y_train shape : ", y_train.shape,"    y_test.shape : ", y_test.shape)
    
    num_features = X_train[0].shape[0]
    #print("number of features : ", num_features)

    #print("Splitting dataset into batch")
    X_batch = np.array_split(X_train, 1)
    y_batch = np.array_split(y_train, 1)

    #print(X_batch[0].shape)
    
    w = []
    i=0
    for y,batch in zip(y_batch,X_batch):
        w.append(implementations.ridge_regression(y, batch,lambda_))
    
    
    wp =  np.sum(np.array(w),axis=0)/(len(X_batch))
    
    
    

    y_pred_ridge_train = proj1_helpers.predict_labels(wp,X_train)
    y_pred_ridge_test = proj1_helpers.predict_labels(wp,X_test)
    s_tr = 0
    s_te = 0
    tot_tr = 0
    for i,y in enumerate(y_train):
        if y == y_pred_ridge_train[i]:
            s_tr += 1
        tot_tr += 1
    tot_te = 0
    for i,y in enumerate(y_test):
        if y == y_pred_ridge_test[i]:
            s_te += 1
        tot_te += 1

    print("Ridge precision on train : ",s_tr/tot_tr,"      Ridge precision on test : ",s_te/tot_te)
    
    return wp

In [16]:
wp_out = batch_ridge_out(1000,7,32)
wp_out = batch_ridge_out(2000,7,32)
wp_out = batch_ridge_out(3000,7,32)
wp_out = batch_ridge_out(5000,7,32)
wp_out = batch_ridge_out(8000,7,32)
wp_out = batch_ridge_out(10000,7,32)
wp_out = batch_ridge_out(20000,7,32)
wp_out = batch_ridge_out(300,7,32)


Ridge precision on train :  0.8117516557371701       Ridge precision on test :  0.8131238990017616
Ridge precision on train :  0.8113438387002055       Ridge precision on test :  0.8128302994715209
Ridge precision on train :  0.8110338977521125       Ridge precision on test :  0.8140046975924838
Ridge precision on train :  0.8108055202114124       Ridge precision on test :  0.8138578978273635
Ridge precision on train :  0.8110012723891553       Ridge precision on test :  0.8138578978273635
Ridge precision on train :  0.810936021663241       Ridge precision on test :  0.8142982971227246
Ridge precision on train :  0.8104466412188835       Ridge precision on test :  0.8126834997064005
Ridge precision on train :  0.8123225995889204       Ridge precision on test :  0.8135642982971227


In [43]:
ytest, input_data_test, ids_test = proj1_helpers.load_csv_data("test.csv")
xsub = []
degree = 7
for i in range(1,degree+1):
    xsub.append(np.power(input_data_test,i))
inp = np.concatenate(xsub,axis = 1)



In [52]:
ypred = []
for i in range(inp.shape[0]):
    x = inp[i]
    if -999 in x:
        ypred.append(np.dot(wp_in,x[:180]))
    else:
        ypred.append(np.dot(wp_out,x))
ypred = np.array(ypred)
ypred[np.where(ypred <= 0)] = -1
ypred[np.where(ypred > 0)] = 1
proj1_helpers.create_csv_submission(ids_test, ypred, "pred4.csv")

In [42]:
s_te = 0
tot_te = 0
for i,y in enumerate(ytest):
    if y == ypred[i]:
        s_te += 1
    tot_te += 1

print("Ridge precision on train : ",s_te/tot_te)#     Ridge precision on test : ",s_te/tot_te)
    

Ridge precision on train :  0.807976


In [23]:
print(ypred)

[-1. -1. -1. ..., -1.  1. -1.]
